# Using Neural Networks for Image Recognition

For this portfolio, we will make use of the Logo Images Dataset obtained from \url{https://github.com/msn199959/Logo-2k-plus-Dataset}, it contains over 150, 000 images, which is

## Introduction

A neural network attempts to replicate the structure of the brain, in which neurons pass electrical current form a directed network. Here if enough neurons sense an the input (for example a touch on the skin), a signal is sent via a charge through the network to produce an output.

An artificial neural network imitates this by having an input layer, hidden layers and an output layer as shown below:

#\begin{tikzpicture}

#\end{tikzpicture}

As we can see, the internal neurons can have multiple inputs and outputs. The inputs, $x_1, ... ,x_n$, have weights $w_1, ... , w_n$ and this weighted input is passed to an activation function $\phi()$, to get the output of the neuron as:
$$
y = \phi \left( \sum_i x_i w_i \right)
$$

A simple neural network is made up of an input layer, hidden layers and an output layers. Our aim is to select correct weights on each edge using iterative methods.

## Backpropogation

This is a training method, also referred to "the backward propogation of errors". To use this, we first define the following quatities

\begin{align}
    J(y) &= (t-y)^2 \text{ the loss function,} \\
    D_n(y) &= \frac{d J(y)}{dw_n} \text{ the derivative of the loss function }
\end{align}

We then perform the following steps for each $(x,t) \in X$
1. Pass $x$ through the neural network and obtain the output $y$
2. Obtain the new weight for each edge $w_n' = \delta w_n = -RD_n(y)$ for a learning rate $R$

## The Iris Dataset

We now train a neural network on the iris dataset

In [2]:
import tensorflow

from sklearn.datasets import load_iris


X, y = load_iris(as_frame=True, return_X_y=True)

We can now get a training and testing set for our network

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
import tensorflow as tf

train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [7]:
test = test.batch(1)

## Creating the Model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu),   # hidden layer
    tf.keras.layers.Dense(10, activation=tf.nn.relu),   # hidden layer
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)  # output layer
])

In [9]:
model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [10]:
model.fit(
    train,
    validation_data=test,
    epochs=10
)

Epoch 1/10


ValueError: in user code:

    File "/home/ac18826/Documents/COMPASS/First_Year/TB2/Statistical_Computing/tfvenv/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/ac18826/Documents/COMPASS/First_Year/TB2/Statistical_Computing/tfvenv/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ac18826/Documents/COMPASS/First_Year/TB2/Statistical_Computing/tfvenv/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/ac18826/Documents/COMPASS/First_Year/TB2/Statistical_Computing/tfvenv/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/ac18826/Documents/COMPASS/First_Year/TB2/Statistical_Computing/tfvenv/lib/pytho